In [1]:
using CSV, DataFrames, Plots, Statistics, StatsBase, JLD, Dates

include("Mods.jl");

datapath = "../Data/";

theme(:dark)
Plots.default(titlefontsize=12, tickfontsize=10, labelfontsize=12, legendfontsize=9,
    fontfamily="Computer Modern", frame=:box, label=nothing)
plotlyjs();

In [2]:
Data = CSV.read(datapath * "Stage0/Data_0.csv", DataFrame);
SelectedProds = load(datapath * "Stage0/SelectedData/SelectedProds.jld")["SelectedProds"];
SelectedData = [CSV.read(datapath * "Stage0/SelectedData/$i.csv", DataFrame) for i ∈ SelectedProds];

In [5]:
for i ∈ 1:length(SelectedProds)
    df = SelectedData[i]
    Prod = SelectedProds[i]
    DateAns = df.Date
    PriceAns = df.Price
    NewDateAns = sort(collect(Set(DateAns)))
    NewPriceAns = [mean(PriceAns[findall(x -> x == d, DateAns)]) for d ∈ NewDateAns]
    NewDF = DataFrame("Date" => NewDateAns, "Price" => NewPriceAns)
    CSV.write(datapath * "Stage1/$Prod.csv", NewDF)
end

In [6]:
StartConDate = 410
EndConDate = 1700
SelectedDates = StartConDate:EndConDate
PricesMatrix = Matrix{Float64}(undef, length(SelectedDates), length(SelectedProds))
PriceDataFrame = DataFrame()
for i ∈ 1:length(SelectedProds)
    dates = SelectedData[i].Date
    Prices = SelectedData[i].Price
    PricesMatrix[:, i] = [CostumMods.FindMeanPrice(Prices, dates, x) for x ∈ SelectedDates]
    PriceDataFrame[!, SelectedProds[i]] = PricesMatrix[:, i]
end
PriceDataFrame.Date = collect(SelectedDates)
CSV.write(datapath * "Stage1/PricesMatrix.csv", PriceDataFrame)
save(datapath * "Stage1/PricesMatrix.jld", "PricesMatrix", PricesMatrix)